In [1]:
using GLFW
using ModernGL
using LinearAlgebra

const alto = 800
const ancho = 800

# 1. INICIALIZAR
GLFW.Init()
ventana = GLFW.CreateWindow(ancho, alto, "Figuras con Color")
GLFW.MakeContextCurrent(ventana)

# 2. SHADERS CON COLOR VARIABLE
vertex_shader_source = """
#version 330 core
layout (location = 0) in vec2 aPos;
void main() {
    gl_Position = vec4(aPos, 0.0, 1.0);
}
"""

fragment_shader_source = """
#version 330 core
out vec4 FragColor;
uniform vec4 nuestroColor;
void main() {
    FragColor = nuestroColor;
}
"""

# Compilar shaders
vertex_shader = glCreateShader(GL_VERTEX_SHADER)
glShaderSource(vertex_shader, 1, [pointer(vertex_shader_source)], C_NULL)
glCompileShader(vertex_shader)

fragment_shader = glCreateShader(GL_FRAGMENT_SHADER)
glShaderSource(fragment_shader, 1, [pointer(fragment_shader_source)], C_NULL)
glCompileShader(fragment_shader)

shader_program = glCreateProgram()
glAttachShader(shader_program, vertex_shader)
glAttachShader(shader_program, fragment_shader)
glLinkProgram(shader_program)

color_location = glGetUniformLocation(shader_program, "nuestroColor")

mutable struct Ray
    vel::Float64
    Position::Vector{Float64}
    direc::Vector{Float64}
    r::Float64
    θ::Float64
    dr::Float64
    dθ::Float64
end

function crear_ray(c, Position, direc)
    r = sqrt(Position[1]^2 + Position[2]^2)
    θ = atan(Position[2], Position[1])
    dr = direc[1] * cos(θ) + direc[2] * sin(θ)
    dθ = (-direc[1] * sin(θ) + direc[2] * cos(θ)) / r
    return Ray(c, Position, direc, r, θ, dr, dθ)
end

function normalized(direc)
    n = norm(direc)
    return [direc[1]/n, direc[2]/n]
end

struct BlackHole
    r::Float64
    Position::Vector{Float64}
    m::Float64
end

function crear_black_hole(m, Position)
    radio = 2 * m
    return BlackHole(radio, Position, m)
end

# 3. DATOS DEL CÍRCULO (agujero negro)
Black = crear_black_hole(30, [0.0, 0.0])

vertices_circulo = Float32[]
for i in 0:36
    angulo = 2π * i / 36
    x = Black.Position[1] + cos(angulo) * Black.r/ancho
    y = Black.Position[2] + sin(angulo) * Black.r/alto
    push!(vertices_circulo, x, y)
end

# 4. CONFIGURACIÓN DE MÚLTIPLES RAYOS DESDE TODOS LOS LADOS
n_rays = 100  # Número de rayos
rayos = []
trayectorias = []
direcciones = []

# Inicializar rayos desde todos los lados
for i in 1:n_rays
    if i <= n_rays ÷ 4
        # Rayos desde la izquierda hacia la derecha
        y_pos = -1.0 + (2.0 * (i-1)) / (n_rays÷4 - 1)
        pos₀ = [-1.0, y_pos]
        dir = normalized([1.0, 0.0])
    elseif i <= n_rays ÷ 2
        # Rayos desde la derecha hacia la izquierda
        idx = i - n_rays ÷ 4
        y_pos = -1.0 + (2.0 * (idx-1)) / (n_rays÷4 - 1)
        pos₀ = [1.0, y_pos]
        dir = normalized([-1.0, 0.0])
    elseif i <= 3 * n_rays ÷ 4
        # Rayos desde abajo hacia arriba
        idx = i - n_rays ÷ 2
        x_pos = -1.0 + (2.0 * (idx-1)) / (n_rays÷4 - 1)
        pos₀ = [x_pos, -1.0]
        dir = normalized([0.0, 1.0])
    else
        # Rayos desde arriba hacia abajo
        idx = i - 3 * n_rays ÷ 4
        x_pos = -1.0 + (2.0 * (idx-1)) / (n_rays÷4 - 1)
        pos₀ = [x_pos, 1.0]
        dir = normalized([0.0, -1.0])
    end
    
    push!(rayos, crear_ray(1.0, pos₀, dir))
    push!(trayectorias, Float32[pos₀[1], pos₀[2]])
    push!(direcciones, dir)
end

# 5. FUNCIONES DE FÍSICA

#elejir metodo y porque cual 
function geodesic_equations(ray::Ray, rs)
    r = ray.r
    dθ = ray.dθ
    dr = ray.dr
    rs = rs / alto  
    
    
    if r > rs
        d2r = r * dθ^2 - (3.0 * rs * dθ^2) - (rs * dr^2) / (2 * r * (r - rs))
        d2θ = -2 * dr * dθ / r
    else
        d2r = -rs / (2 * r^2)
        d2θ = 0.0
    end
    
    return d2r, d2θ
end

function step!(ray::Ray, dt, R_s)
    d2r, d2θ = geodesic_equations(ray, R_s)
    
    ray.dr += dt * d2r
    ray.dθ += dt * d2θ
    
    ray.r += dt * ray.dr
    ray.θ += dt * ray.dθ
    
    x = ray.r * cos(ray.θ)
    y = ray.r * sin(ray.θ)
    
    ray.Position = [x, y]
    
    # Reset si se sale de la pantalla o cae en el agujero negro
    if abs(x) > 2.0 || abs(y) > 2.0 || ray.r <= R_s/ancho
        return true
    end
    
    return false
end

# 6. BUFFERS PARA MÚLTIPLES RAYOS
VAO_circulo = zeros(UInt32, 1)
VBO_circulo = zeros(UInt32, 1)

# Arrays para VAOs y VBOs de trayectorias y puntos
VAO_trayectorias = zeros(UInt32, n_rays)
VBO_trayectorias = zeros(UInt32, n_rays)
VAO_puntos = zeros(UInt32, n_rays)
VBO_puntos = zeros(UInt32, n_rays)

glGenVertexArrays(1, VAO_circulo)
glGenBuffers(1, VBO_circulo)
glGenVertexArrays(n_rays, VAO_trayectorias)
glGenBuffers(n_rays, VBO_trayectorias)
glGenVertexArrays(n_rays, VAO_puntos)
glGenBuffers(n_rays, VBO_puntos)

# Configurar círculo (agujero negro)
glBindVertexArray(VAO_circulo[1])
glBindBuffer(GL_ARRAY_BUFFER, VBO_circulo[1])
glBufferData(GL_ARRAY_BUFFER, sizeof(vertices_circulo), vertices_circulo, GL_STATIC_DRAW)
glVertexAttribPointer(0, 2, GL_FLOAT, GL_FALSE, 2 * sizeof(Float32), C_NULL)
glEnableVertexAttribArray(0)

# Configurar buffers para cada rayo
for i in 1:n_rays
    # Configurar trayectoria
    glBindVertexArray(VAO_trayectorias[i])
    glBindBuffer(GL_ARRAY_BUFFER, VBO_trayectorias[i])
    glVertexAttribPointer(0, 2, GL_FLOAT, GL_FALSE, 2 * sizeof(Float32), C_NULL)
    glEnableVertexAttribArray(0)
    
    # Configurar punto
    glBindVertexArray(VAO_puntos[i])
    glBindBuffer(GL_ARRAY_BUFFER, VBO_puntos[i])
    glVertexAttribPointer(0, 2, GL_FLOAT, GL_FALSE, 2 * sizeof(Float32), C_NULL)
    glEnableVertexAttribArray(0)
end

# 7. LOOP PRINCIPAL
while !GLFW.WindowShouldClose(ventana)
    glClearColor(0.0, 0.0, 0.0, 1.0)
    glClear(GL_COLOR_BUFFER_BIT)
    
    glUseProgram(shader_program)
    
    # Dibujar agujero negro (rojo)
    glUniform4f(color_location, 1.0, 0.0, 0.0, 1.0)
    glBindVertexArray(VAO_circulo[1])
    glDrawArrays(GL_TRIANGLE_FAN, 0, length(vertices_circulo) ÷ 2)
    
    # Actualizar y dibujar cada rayo
    dt = 0.0001
    for i in 1:n_rays
        # Actualizar física
        needs_reset = step!(rayos[i], dt, Black.r)
        
        if needs_reset
            # Resetear a posición inicial según la dirección original
            if i <= n_rays ÷ 4
                # Rayos desde la izquierda
                idx = i
                y_pos = -1.0 + (2.0 * (idx-1)) / (n_rays÷4 - 1)
                pos₀ = [-1.0, y_pos]
                dir = normalized([1.0, 0.0])
            elseif i <= n_rays ÷ 2
                # Rayos desde la derecha
                idx = i - n_rays ÷ 4
                y_pos = -1.0 + (2.0 * (idx-1)) / (n_rays÷4 - 1)
                pos₀ = [1.0, y_pos]
                dir = normalized([-1.0, 0.0])
            elseif i <= 3 * n_rays ÷ 4
                # Rayos desde abajo
                idx = i - n_rays ÷ 2
                x_pos = -1.0 + (2.0 * (idx-1)) / (n_rays÷4 - 1)
                pos₀ = [x_pos, -1.0]
                dir = normalized([0.0, 1.0])
            else
                # Rayos desde arriba
                idx = i - 3 * n_rays ÷ 4
                x_pos = -1.0 + (2.0 * (idx-1)) / (n_rays÷4 - 1)
                pos₀ = [x_pos, 1.0]
                dir = normalized([0.0, -1.0])
            end
            
            rayos[i] = crear_ray(1.0, pos₀, dir)
            trayectorias[i] = Float32[pos₀[1], pos₀[2]]
        else
            # Agregar nueva posición a la trayectoria
            push!(trayectorias[i], Float32(rayos[i].Position[1]), Float32(rayos[i].Position[2]))
        end
        
        # Dibujar trayectoria (gris)
        if length(trayectorias[i]) > 2
            glBindVertexArray(VAO_trayectorias[i])
            glBindBuffer(GL_ARRAY_BUFFER, VBO_trayectorias[i])
            glBufferData(GL_ARRAY_BUFFER, sizeof(trayectorias[i]), trayectorias[i], GL_DYNAMIC_DRAW)
            glUniform4f(color_location, 0.5, 0.5, 0.5, 1.0)
            glDrawArrays(GL_LINE_STRIP, 0, length(trayectorias[i]) ÷ 2)
        end
        
        # Dibujar posición actual (blanco)
        vertices_punto = Float32[rayos[i].Position[1], rayos[i].Position[2]]
        glBindVertexArray(VAO_puntos[i])
        glBindBuffer(GL_ARRAY_BUFFER, VBO_puntos[i])
        glBufferData(GL_ARRAY_BUFFER, sizeof(vertices_punto), vertices_punto, GL_DYNAMIC_DRAW)
        glUniform4f(color_location, 1.0, 1.0, 1.0, 1.0)
        glPointSize(3.0)
        glDrawArrays(GL_POINTS, 0, 1)
    end
    
    GLFW.SwapBuffers(ventana)
    GLFW.PollEvents()
end

# 8. LIMPIAR
glDeleteVertexArrays(1, VAO_circulo)
glDeleteBuffers(1, VBO_circulo)
glDeleteVertexArrays(n_rays, VAO_trayectorias)
glDeleteBuffers(n_rays, VBO_trayectorias)
glDeleteVertexArrays(n_rays, VAO_puntos)
glDeleteBuffers(n_rays, VBO_puntos)
glDeleteProgram(shader_program)
GLFW.Terminate()

In [2]:
using GLFW
using ModernGL
using LinearAlgebra

const alto = 800
const ancho = 800

# 1. INICIALIZAR
GLFW.Init()
ventana = GLFW.CreateWindow(ancho, alto, "Figuras con Color")
GLFW.MakeContextCurrent(ventana)

# 2. SHADERS 3D
vertex_shader_source = """
#version 330 core
layout (location = 0) in vec3 aPos;  // CAMBIO: vec3 en lugar de vec2
void main() {
    gl_Position = vec4(aPos, 1.0);   // Mantenemos proyección simple
}
"""

fragment_shader_source = """
#version 330 core
out vec4 FragColor;
uniform vec4 nuestroColor;
void main() {
    FragColor = nuestroColor;
}
"""

# Compilar shaders
vertex_shader = glCreateShader(GL_VERTEX_SHADER)
glShaderSource(vertex_shader, 1, [pointer(vertex_shader_source)], C_NULL)
glCompileShader(vertex_shader)

fragment_shader = glCreateShader(GL_FRAGMENT_SHADER)
glShaderSource(fragment_shader, 1, [pointer(fragment_shader_source)], C_NULL)
glCompileShader(fragment_shader)

shader_program = glCreateProgram()
glAttachShader(shader_program, vertex_shader)
glAttachShader(shader_program, fragment_shader)
glLinkProgram(shader_program)

color_location = glGetUniformLocation(shader_program, "nuestroColor")

# ESTRUCTURAS 3D
mutable struct Ray
    vel::Float64
    Position::Vector{Float64}  # [x, y, z]
    direc::Vector{Float64}     # [dx, dy, dz]  
    r::Float64
    θ::Float64
    ϕ::Float64                 # NUEVO - ángulo azimutal
    dr::Float64
    dθ::Float64
    dϕ::Float64                # NUEVO
end

struct BlackHole
    r::Float64
    Position::Vector{Float64}  # [x, y, z]
    m::Float64
end

function crear_black_hole(m, Position)
    radio = 2 * m
    # Asegurar que Position sea 3D
    if length(Position) == 2
        Position = [Position[1], Position[2], 0.0]  # Añadir z=0
    end
    return BlackHole(radio, Position, m)
end

function crear_ray(c, Position, direc)
    x, y, z = Position
    r = sqrt(x^2 + y^2 + z^2)
    θ = acos(z / r)                    # ángulo polar
    ϕ = atan(y, x)                     # ángulo azimutal
    
    # Conversión de velocidades a coordenadas esféricas
    dr = (direc[1]*x + direc[2]*y + direc[3]*z) / r
    dθ = (z*dr - r*direc[3]) / (r^2 * sqrt(1 - (z/r)^2))
    dϕ = (-direc[1]*y + direc[2]*x) / (x^2 + y^2)
    
    return Ray(c, Position, direc, r, θ, ϕ, dr, dθ, dϕ)
end

function normalized(direc)
    n = norm(direc)
    return [direc[1]/n, direc[2]/n, direc[3]/n]
end

# 3. DATOS DEL CÍRCULO (agujero negro) - Mantenemos visualización 2D
Black = crear_black_hole(30, [0.0, 0.0])

vertices_circulo = Float32[]
for i in 0:36
    angulo = 2π * i / 36
    x = Black.Position[1] + cos(angulo) * Black.r/ancho
    y = Black.Position[2] + sin(angulo) * Black.r/alto
    push!(vertices_circulo, x, y)
end

# 4. CONFIGURACIÓN DE MÚLTIPLES RAYOS 3D
n_rays = 16
rayos = []
trayectorias = []
direcciones = []

# Inicializar rayos en plano z=0
for i in 1:n_rays
    y_pos = -1.0 + (2.0 * (i-1)) / (n_rays-1)
    pos₀ = [-1.0, y_pos, 0.0]  # z = 0.0
    dir = normalized([1.0, 0.0, 0.0])  # Movimiento en X
        
    push!(rayos, crear_ray(1.0, pos₀, dir))
    push!(trayectorias, Float32[pos₀[1], pos₀[2], pos₀[3]])  # 3 componentes
    push!(direcciones, dir)
end

# 5. FÍSICA 3D - SCHWARZSCHILD
function geodesic_equations(ray::Ray, rs)
    r = ray.r
    θ = ray.θ
    dr = ray.dr
    dθ = ray.dθ
    dϕ = ray.dϕ
    
    rs = rs / alto  # Misma escala que antes
    
    if r > rs
        # Ecuaciones geodésicas Schwarzschild 3D
        d2r = r * (dθ^2 + sin(θ)^2 * dϕ^2) - (rs * dθ^2) - (rs * sin(θ)^2 * dϕ^2)
        d2θ = -2.0 * dr * dθ / r + sin(θ) * cos(θ) * dϕ^2
        d2ϕ = -2.0 * dr * dϕ / r - 2.0 * cos(θ) * dθ * dϕ / sin(θ)
    else
        # Dentro del horizonte
        d2r = -rs / (2.0 * r^2)
        d2θ = 0.0
        d2ϕ = 0.0
    end
    
    return d2r, d2θ, d2ϕ
end

function step!(ray::Ray, dt, R_s)
    d2r, d2θ, d2ϕ = geodesic_equations(ray, R_s)
    
    ray.dr += dt * d2r
    ray.dθ += dt * d2θ
    ray.dϕ += dt * d2ϕ
    
    ray.r += dt * ray.dr
    ray.θ += dt * ray.dθ
    ray.ϕ += dt * ray.dϕ
    
    # Convertir de esféricas a cartesianas 3D
    x = ray.r * sin(ray.θ) * cos(ray.ϕ)
    y = ray.r * sin(ray.θ) * sin(ray.ϕ)
    z = ray.r * cos(ray.θ)
    
    ray.Position = [x, y, z]
    
    # Misma condición de reset
    if abs(x) > 2.0 || abs(y) > 2.0 || ray.r <= R_s/ancho
        return true
    end
    
    return false
end

# 6. BUFFERS 3D
VAO_circulo = zeros(UInt32, 1)
VBO_circulo = zeros(UInt32, 1)

VAO_trayectorias = zeros(UInt32, n_rays)
VBO_trayectorias = zeros(UInt32, n_rays)
VAO_puntos = zeros(UInt32, n_rays)
VBO_puntos = zeros(UInt32, n_rays)

glGenVertexArrays(1, VAO_circulo)
glGenBuffers(1, VBO_circulo)
glGenVertexArrays(n_rays, VAO_trayectorias)
glGenBuffers(n_rays, VBO_trayectorias)
glGenVertexArrays(n_rays, VAO_puntos)
glGenBuffers(n_rays, VBO_puntos)

# Configurar círculo (agujero negro) - sigue siendo 2D para visualización
glBindVertexArray(VAO_circulo[1])
glBindBuffer(GL_ARRAY_BUFFER, VBO_circulo[1])
glBufferData(GL_ARRAY_BUFFER, sizeof(vertices_circulo), vertices_circulo, GL_STATIC_DRAW)
glVertexAttribPointer(0, 2, GL_FLOAT, GL_FALSE, 2 * sizeof(Float32), C_NULL)  # Mantenemos 2D para el círculo
glEnableVertexAttribArray(0)

# Configurar buffers para cada rayo - AHORA 3D
for i in 1:n_rays
    # Configurar trayectoria
    glBindVertexArray(VAO_trayectorias[i])
    glBindBuffer(GL_ARRAY_BUFFER, VBO_trayectorias[i])
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 3 * sizeof(Float32), C_NULL)  # CAMBIO: 3 componentes
    glEnableVertexAttribArray(0)
    
    # Configurar punto
    glBindVertexArray(VAO_puntos[i])
    glBindBuffer(GL_ARRAY_BUFFER, VBO_puntos[i])
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 3 * sizeof(Float32), C_NULL)  # CAMBIO: 3 componentes
    glEnableVertexAttribArray(0)
end

# 7. LOOP PRINCIPAL
while !GLFW.WindowShouldClose(ventana)
    glClearColor(0.0, 0.0, 0.0, 1.0)
    glClear(GL_COLOR_BUFFER_BIT)
    
    glUseProgram(shader_program)
    
    # Dibujar agujero negro (rojo)
    glUniform4f(color_location, 1.0, 0.0, 0.0, 1.0)
    glBindVertexArray(VAO_circulo[1])
    glDrawArrays(GL_TRIANGLE_FAN, 0, length(vertices_circulo) ÷ 2)
    
    # Actualizar y dibujar cada rayo
    dt = 0.0001
    for i in 1:n_rays
        # Actualizar física 3D
        needs_reset = step!(rayos[i], dt, Black.r)
        
        if needs_reset
            # Resetear a posición inicial
            y_pos = -1.0 + (2.0 * (i-1)) / (n_rays-1)
            pos₀ = [-1.0, y_pos, 0.0]
            rayos[i] = crear_ray(1.0, pos₀, direcciones[i])
            trayectorias[i] = Float32[pos₀[1], pos₀[2], pos₀[3]]
        else
            # Agregar nueva posición a la trayectoria
            push!(trayectorias[i], Float32(rayos[i].Position[1]), Float32(rayos[i].Position[2]), Float32(rayos[i].Position[3]))
        end
        
        # Dibujar trayectoria (gris)
        if length(trayectorias[i]) > 3  # Cambio: 3 en lugar de 2
            glBindVertexArray(VAO_trayectorias[i])
            glBindBuffer(GL_ARRAY_BUFFER, VBO_trayectorias[i])
            glBufferData(GL_ARRAY_BUFFER, sizeof(trayectorias[i]), trayectorias[i], GL_DYNAMIC_DRAW)
            glUniform4f(color_location, 0.5, 0.5, 0.5, 1.0)
            glDrawArrays(GL_LINE_STRIP, 0, length(trayectorias[i]) ÷ 3)  # Cambio: ÷ 3
        end
        
        # Dibujar posición actual (blanco)
        vertices_punto = Float32[rayos[i].Position[1], rayos[i].Position[2], rayos[i].Position[3]]  # 3 componentes
        glBindVertexArray(VAO_puntos[i])
        glBindBuffer(GL_ARRAY_BUFFER, VBO_puntos[i])
        glBufferData(GL_ARRAY_BUFFER, sizeof(vertices_punto), vertices_punto, GL_DYNAMIC_DRAW)
        glUniform4f(color_location, 1.0, 1.0, 1.0, 1.0)
        glPointSize(3.0)
        glDrawArrays(GL_POINTS, 0, 1)
    end
    
    GLFW.SwapBuffers(ventana)
    GLFW.PollEvents()
end

# 8. LIMPIAR
glDeleteVertexArrays(1, VAO_circulo)
glDeleteBuffers(1, VBO_circulo)
glDeleteVertexArrays(n_rays, VAO_trayectorias)
glDeleteBuffers(n_rays, VBO_trayectorias)
glDeleteVertexArrays(n_rays, VAO_puntos)
glDeleteBuffers(n_rays, VBO_puntos)
glDeleteProgram(shader_program)
GLFW.Terminate()

ErrorException: invalid redefinition of constant Main.Ray